In [1]:
from climsim_utils.data_utils import *
from create_dataset_example_v5expandcnv_import import read_save_data

2025-04-11 00:01:26.010783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 00:01:28.119974: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-11 00:01:28.390063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-11 00:01:28.900350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-11 00:01:29.102486: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 00:01:30.062139: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# npdata = np.load('/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandrad_postprocessed/train_input.npy')
# # npdata2 = np.load('/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandrad_postprocessed/train_input.h5')
# npdata.shape, npdata.shape[0]/21600

# # import matplotlib.pyplot as plt
# # plt.hist(npdata[:,59])

### Instantiating class

The example below will save training data in both .h5 and .npy format. Adjust if you only need one format. Also adjust input_abbrev to the input data files you will use. We expanded the original '.mli.' input files to include additional features such as previous steps' information, and '.mlexpand.' was just an arbitrary name we used for the expanded input files.

Currently the training script would assume the training set is in .h5 format while the validation set is in .npy form. It's fine to only keep save_h5=True in the block below for generating training data.

In [3]:
# grid_path = '/global/u2/z/zeyuanhu/nvidia_codes/Climsim_private/grid_info/ClimSim_low-res_grid-info.nc'
# norm_path = '/global/u2/z/zeyuanhu/nvidia_codes/Climsim_private/preprocessing/normalizations/'

grid_path = '/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/ClimSim_high-res_grid-info.nc'
norm_path = '/p/project/icon-a-ml/heuer1/ClimSim/preprocessing/normalizations/'

grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + 'inputs/input_mean_v5_pervar.nc')
input_max = xr.open_dataset(norm_path + 'inputs/input_max_v5_pervar.nc')
input_min = xr.open_dataset(norm_path + 'inputs/input_min_v5_pervar.nc')
output_scale = xr.open_dataset(norm_path + 'outputs/output_scale_std_lowerthred_v5.nc')

data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale,
                  input_abbrev = 'mlexpandcnv',
                  output_abbrev = 'mlo',
                  #ml_backend = 'pytorch',
                  normalize=False,
                  save_h5=False,
                  save_zarr=True,
                  save_npy=False
                  )

In [4]:
# set data path
# data.data_path = '/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/train/'
data.data_path = '/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/train/'

# set inputs and outputs to V5 subset
data.set_to_v5cnvqcqi_vars()

In [5]:
data.input_vars, data.target_vars

(['state_t',
  'state_rh',
  'state_q0002',
  'state_q0003',
  'liq_partition',
  'state_u',
  'state_v',
  'state_t_prvphy',
  'state_q0001_prvphy',
  'state_q0002_prvphy',
  'state_q0003_prvphy',
  'state_u_prvphy',
  'tm_state_t_prvphy',
  'tm_state_q0001_prvphy',
  'tm_state_q0002_prvphy',
  'tm_state_q0003_prvphy',
  'tm_state_u_prvphy',
  'state_pmid',
  'state_ps',
  'pbuf_LHFLX',
  'pbuf_SHFLX',
  'pbuf_TAUX',
  'pbuf_TAUY',
  'pbuf_COSZRS',
  'cam_in_ALDIF',
  'cam_in_ALDIR',
  'cam_in_ASDIF',
  'cam_in_ASDIR',
  'tm_state_ps',
  'tm_pbuf_LHFLX',
  'tm_pbuf_SHFLX',
  'tm_pbuf_COSZRS',
  'clat',
  'slat'],
 ['ptend_t',
  'ptend_q0001',
  'ptend_q0002',
  'ptend_q0003',
  'ptend_u',
  'ptend_v',
  'cam_out_PRECSC',
  'cam_out_PRECC'])

### Create training data

Below is an example of creating the training data by integrating the 7 year climsim simulation data. A subsampling of 1000 is used as an example. In the actual work we did, we used a stride_sample=1. We could not fit the full 7-year data into the memory wihout subsampling. If that's also the case for you, try to only process a subset of data at one time by adjusting regexps in set_regexps method. We saved 14 separate input .h5 files. For each year, we saved two files by setting start_idx=0 or 1. For each year, we saved two files by setting start_idx=0 or 1. We have a folder like v5_full, which includes 14 subfolders named '11', '12', '21', '22', ..., '71','72', and each subfolder contains a train_input.h5 and train_target.h5. How you split to save training data won't influence the training. The training script will read in all the samples and randomly select samples across all the samples to form each batch.

In [6]:
# # default: ignore loop (one iteration)
# split='train'
# # regex_list = [f'E3SM-MMF.mlexpandcnv.0001-*-*.nc', 'E3SM-MMF.mlexpandcnv.0002-01-*-*.nc']
# regex_list = [f'E3SM-MMF.mlexpandcnv.0001-{i:02d}-*.nc' for i in range(2, 13)] + ['E3SM-MMF.mlexpandcnv.0002-01-*-*.nc']
# # split='val'
# # regex_list = [f'E3SM-MMF.mlexpandcnv.0002-06-10-*.nc']
# # split='test'
# # regex_list = [f'E3SM-MMF.mlexpandcnv.0002-06-20-*.nc']

# for i in tqdm(range(0,len(regex_list))):#-1,-1,-1)):#
#         # set regular expressions for selecting training data
#         # data.set_regexps(data_split = 'train',
#         #                 regexps = [f'E3SM-MMF.mlexpandcnv.000[1234567]-*-*.nc',#{i:02d}-*.nc', # years 1 through 7
#         #                         'E3SM-MMF.mlexpandcnv.0008-01-*-*.nc']) # first month of year 8
#         data.set_regexps(data_split = split,
#                          # regexps = regex_list)
#                          regexps = [regex_list[i]])
#         # set temporal subsampling
#         data.set_stride_sample(data_split = split,stride_sample = 1)
#         # create list of files to extract data from
#         data.set_filelist(data_split = split, start_idx=0)
#         # print(data.train_filelist)
#         # print(len(sorted(data.val_filelist))
#         # save numpy files of training data
#         # print(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnvcons_postprocessed/{i:02}')
#         # save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnvcons_postprocessed/'
#         # print(i)
#         # save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi/'
#         save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/{i:02}/'
#         os.makedirs(save_path)#, exist_ok=True)
#         data.save_as_npy(data_split = split, save_path = save_path)

In [7]:
mlexpandcnv_paths = glob.glob(f'{data.data_path}/**/E3SM-MMF.{data.input_abbrev}.*', recursive=True)
yms = sorted(list(set([re.search(f'.*E3SM-MMF\.{data.input_abbrev}\.(\d\d\d\d-\d\d)-.*\.nc', p).group(1) for p in mlexpandcnv_paths])))
len(yms)

96

## mp

In [ ]:
import multiprocessing as mp

mp.set_start_method('spawn')

# test = []
if __name__ == '__main__':
    # Determine the number of processes based on system's capabilities or your preference
    num_processes = mp.cpu_count()  # You can adjust this to a fixed number if preferred
    print(num_processes)
    num_processes = 24
    args_for_processing = [(ym, 'train', '/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_qcqi') for ym in yms]
    with mp.Pool(num_processes) as pool:
        # Use pool.map to process files in parallel
        pool.map(read_save_data, args_for_processing)

In [8]:
exist_files = os.listdir('/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_qcqi')
yms = [ym for ym in yms if ym not in exist_files]
yms

['0003-12']

In [9]:
if __name__ == '__main__':
    # Determine the number of processes based on system's capabilities or your preference
    args_for_processing = [(ym, 'train', '/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_qcqi') for ym in yms]
    for arg in tqdm(args_for_processing[:]):
        read_save_data(arg)

  0%|                                                                                                    | 0/1 [00:00<?, ?it/s]

Reading data for 0003-12


2025-04-11 00:02:54.500173: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-04-11 00:11:35.541528: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [09:28<00:00, 568.30s/it]

Finished processing of /p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_qcqi/0003-12


In [ ]:
raise Exception("Stop here in script mode")

In [ ]:
!ls /p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_qcqi

In [ ]:
# default: ignore loop (one iteration)
split='train'
# regex_list = [f'E3SM-MMF.mlexpandcnv.0001-*-*.nc', 'E3SM-MMF.mlexpandcnv.0002-01-*-*.nc']
regex_list = [f'E3SM-MMF.mlexpandcnv.0001-{i:02d}-*.nc' for i in range(2, 13)] + ['E3SM-MMF.mlexpandcnv.0002-01-*-*.nc']
# split='val'
# regex_list = [f'E3SM-MMF.mlexpandcnv.0002-06-10-*.nc']
# split='test'
# regex_list = [f'E3SM-MMF.mlexpandcnv.0002-06-20-*.nc']

for i in tqdm(range(0,len(regex_list))):#-1,-1,-1)):#
        # set regular expressions for selecting training data
        # data.set_regexps(data_split = 'train',
        #                 regexps = [f'E3SM-MMF.mlexpandcnv.000[1234567]-*-*.nc',#{i:02d}-*.nc', # years 1 through 7
        #                         'E3SM-MMF.mlexpandcnv.0008-01-*-*.nc']) # first month of year 8
        data.set_regexps(data_split = split,
                         # regexps = regex_list)
                         regexps = [regex_list[i]])
        # set temporal subsampling
        data.set_stride_sample(data_split = split,stride_sample = 1)
        # create list of files to extract data from
        data.set_filelist(data_split = split, start_idx=0)
        # print(data.train_filelist)
        # print(len(sorted(data.val_filelist))
        # save numpy files of training data
        # print(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnvcons_postprocessed/{i:02}')
        # save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnvcons_postprocessed/'
        # print(i)
        # save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi/'
        save_path = f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/{i:02}/'
        os.makedirs(save_path)#, exist_ok=True)
        data.save_as_npy(data_split = split, save_path = save_path)

## end mp

In [7]:
# For merging chunks
from glob import glob
import dask.array as da
import h5py
import shutil

# in_tar = 'target'

for in_tar in ['input', 'target']:
    pathlist = sorted(glob(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/**/train_{in_tar}.zarr', recursive=True))
    print(len(pathlist))

    datalist = []
    for p in tqdm(pathlist):
        print(p)
        # datalist.append(np.load(p))
        datalist.append(da.from_zarr(p))

    # nparray = np.concatenate(datalist)
    # print(nparray.shape)
    daarray = da.concatenate(datalist)
    print(daarray.shape)
    daarray.rechunk().to_zarr(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/train_{in_tar}.zarr')

# np.save(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed/train_input.npy', nparray)

# # with h5py.File(f'/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed/train_target.h5', 'w') as hdf:
# #     hdf.create_dataset('data', data=nparray, dtype=nparray.dtype)

# del nparray, datalist

12


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 176.82it/s]

/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/00/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/01/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/02/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/03/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/04/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/05/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/06/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/07/train_input.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/08/train_input.zarr
/p/scratch/icon-a-m

12


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1621.46it/s]

/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/00/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/01/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/02/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/03/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/04/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/05/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/06/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/07/train_target.zarr
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/08/train_target.zarr
/p/scratch

In [9]:
for p in tqdm(pathlist):
    dirname = os.path.dirname(p)
    print(dirname)
    shutil.rmtree(dirname)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 61.79it/s]

/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/00
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/01
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/02
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/03
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/04
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/05
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/06
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/07
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/08
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcqi_radfix/09
/p/scratch/icon-a-ml/heuer1/LEAP/ClimSim_high-res/expandcnv_postprocessed_newqcq

### Create validation data

In [13]:
# set regular expressions for selecting validation data
data.set_regexps(data_split = 'val',
                 regexps = ['E3SM-MMF.mlexpand.0008-0[23456789]-*-*.nc', # months 2 through 9 of year 8
                            'E3SM-MMF.mlexpand.0008-1[012]-*-*.nc', # months 10 through 12 of year 8
                            'E3SM-MMF.mlexpand.0009-01-*-*.nc']) # first month of year 9
# set temporal subsampling
# data.set_stride_sample(data_split = 'val', stride_sample = 7)
data.set_stride_sample(data_split = 'val', stride_sample = 700)
# create list of files to extract data from
data.set_filelist(data_split = 'val')
# save numpy files of validation data
data.save_as_npy(data_split = 'val', save_path = '/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/preprocessing/v5_example/')

### Create test data

In [16]:
data.data_path = '/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/_test/'

data.set_to_v5_vars()

# set regular expressions for selecting validation data
data.set_regexps(data_split = 'test',
                 regexps = ['E3SM-MMF.mlexpand.0009-0[3456789]-*-*.nc', 
                            'E3SM-MMF.mlexpand.0009-1[012]-*-*.nc',
                            'E3SM-MMF.mlexpand.0010-*-*-*.nc',
                            'E3SM-MMF.mlexpand.0011-0[12]-*-*.nc'])
# set temporal subsampling
# data.set_stride_sample(data_split = 'test', stride_sample = 7)
data.set_stride_sample(data_split = 'test', stride_sample = 700)
# create list of files to extract data from
data.set_filelist(data_split = 'test')
# save numpy files of validation data
data.save_as_npy(data_split = 'test', save_path = '/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/preprocessing/v5_example/')

In [17]:
!ls /global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/preprocessing/v5_example/

test_input.h5	test_target.npy  train_target.h5   val_input.npy
test_input.npy	train_input.h5	 train_target.npy  val_target.h5
test_target.h5	train_input.npy  val_input.h5	   val_target.npy
